In [ ]:
import pygame
import sys
import time
import random
import heapq

# Initialiser pygame
pygame.init()

# Définir les dimensions de la fenêtre
WIDTH, HEIGHT = 600, 600
TILE_SIZE = WIDTH // 3
FONT_SIZE = 50

goal = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, None]
]

move = [(-1, 0), (1, 0), (0, 1), (0, -1)]

# Couleurs
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 128, 255)

# Initialiser la fenêtre
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Puzzle à glissement 3x3")

# Charger la police
font = pygame.font.Font(None, FONT_SIZE)

#--------------------------HEURISTIQUE-----------------------------
def final_rowcol(number):
    for row in range(3):
        for col in range(3):
            if goal[row][col] == number:
                return row, col

def heuristique(grid):
    somme = 0
    for row in range(3):
        for col in range(3):
            if grid[row][col] is not None:
                # somme = somme + (abs((row + col) - manhattan(grid[row][col])))
                # print(f"{grid[row][col]}:{(abs(row - final_row(grid[row][col])) + abs(col - final_col(grid[row][col])))}")
                final_row, final_col = final_rowcol(grid[row][col])
                somme = somme + (abs(row - final_row) + abs(col - final_col))
    return somme

#--------------------------VOISIN-----------------------------
def find_empty(grid):
    for row in range(3):
        for col in range(3):
            if grid[row][col] is None:
                return row, col

def change_tuile(grid_changed, row_empty, col_empty, row, col):
    grid_changed[row_empty][col_empty], grid_changed[row][col] = grid_changed[row][col], grid_changed[row_empty][col_empty]
    

def find_neighbour(grid):
    neighbour = []
    grid_changed = []
    # find null
    row_empty, col_empty = find_empty(grid)
    for mx, my in move:
        nx, ny = row_empty + mx, col_empty + my
        if 0 <= nx <= 2 and 0 <= ny <= 2 :
            grid_changed = [row[:] for row in grid]
            change_tuile(grid_changed,row_empty,col_empty,nx,ny)
            neighbour.append(grid_changed)
    return neighbour

def astar(grid, OPEN_LIST, CLOSED_LIST, distance):
    liste = []
    grid_opti = []
    # for row in OPEN_LIST:
    #     liste.append(row[1] + row[2])
    # index_min = liste.index(min(liste))
    # grid_opti = OPEN_LIST[index_min][0]
    # CLOSED_LIST.append(grid_opti)

    # for visited in OPEN_LIST:
    #     if visited[0] == grid_opti:
    #         OPEN_LIST.remove(visited)

    neighbour = find_neighbour(grid)
    # print(f"neighbour")
    for row in neighbour:
        # print(f"ROW: {row}")
        # print(f"CLOSED_LIST: {CLOSED_LIST}")
        if row not in CLOSED_LIST:
            OPEN_LIST.append((row, heuristique(row), distance))
            # print(f"OPEN_LIST {OPEN_LIST}")

    if OPEN_LIST:
        for row in OPEN_LIST:
            liste.append(row[1] + row[2])
        index_min = liste.index(min(liste))
        grid_opti = OPEN_LIST[index_min]
        # print(f"grid_opti: {grid_opti}")
        CLOSED_LIST.append(grid_opti[0])
        distance = grid_opti[2]

    for visited in OPEN_LIST:
        if visited[0] in CLOSED_LIST:
            OPEN_LIST.remove(visited)

    
            
    return grid_opti[0], distance

def is_solved(grid):
    return grid == goal

# Dessiner la grille
def draw_grid(grid):
    screen.fill(WHITE)
    for row in range(3):
        for col in range(3):
            tile = grid[row][col]
            x, y = col * TILE_SIZE, row * TILE_SIZE
            if tile is not None:
                pygame.draw.rect(screen, BLUE, (x, y, TILE_SIZE, TILE_SIZE))
                text = font.render(str(tile), True, WHITE)
                text_rect = text.get_rect(center=(x + TILE_SIZE // 2, y + TILE_SIZE // 2))
                screen.blit(text, text_rect)
            pygame.draw.rect(screen, BLACK, (x, y, TILE_SIZE, TILE_SIZE), 2)


def main():
    grid = [
        [4, 1, 3],
        [2, 6, 8],
        [7, 5, None]
    ]
    
    OPEN_LIST = []
    # CLOSED_LIST = set()
    CLOSED_LIST = []
    distance = 0
    # OPEN_LIST.append((grid, heuristique(grid), distance))
    CLOSED_LIST.append(grid)
    clock = pygame.time.Clock()
    # print(f"OPEN_LIST1 {OPEN_LIST}")
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
        grid, distance = astar(grid, OPEN_LIST, CLOSED_LIST, distance)
        # print(f"Distance: {distance} GRID: {grid}")
        print(f"CLOSED_LIST: {CLOSED_LIST}")
        print(f"OPEN_LIST: {OPEN_LIST}")
        if is_solved(grid):
            print("Puzzle solved") 
            break
        distance+=1
        time.sleep(3)
        draw_grid(grid)
        pygame.display.flip()
        clock.tick(30)

if __name__ == "__main__":
    main()